# Dataset

> Scripts to build the different datasets used for modeling

In [ ]:
#| default_exp dataset

In [ ]:
#| export
#| hide
import os
import h5py
import numpy as np
import pandas as pd
from typing import Tuple, Optional, List, Dict
from orbit_generation.data import load_orbit_data, get_orbit_features

2024-05-15 08:35:01.712865: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-15 08:35:01.735731: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-15 08:35:01.735749: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-15 08:35:01.736262: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-15 08:35:01.739678: I tensorflow/core/platform/cpu_feature_guar

## Exact Periods with Time

### All 5 periods

In [ ]:
#| export
def get_5p_em_dataset(data_directory: Optional[str] = '../data', 
                      output_file_path: Optional[str] = '../data/5p_dataset_em.npy') -> Tuple[np.memmap, np.ndarray]:
    """
    Load orbit data and corresponding labels. Optionally, save the loaded data.
    If the output file exists, return the data as a memory-mapped array, otherwise process and save the data.
    After saving, the data is also returned as a memory-mapped array.
    """
    # Extract file extension to determine the type
    _, file_extension = os.path.splitext(output_file_path)

    # Validate supported file types
    if file_extension not in ['.hdf5', '.npy']:
        raise ValueError("Unsupported file extension. Supported extensions are '.hdf5' or '.npy'.")

    # Define the path for labels file
    labels_path = f"{os.path.splitext(output_file_path)[0]}_labels.npy"

    # Check if the combined data file already exists
    if os.path.exists(output_file_path):
        # Load data as memmap
        data_memmap = np.load(output_file_path, mmap_mode='r+') if file_extension == '.npy' else None  # Add hdf5 handling if needed
        labels_memmap = np.load(labels_path)
        return data_memmap, labels_memmap

    # Paths to data files
    orbits_file_path = os.path.join(data_directory, "em_orbits.h5")
    features_file_path = os.path.join(data_directory, "em_features.mat")

    # Load orbit labels
    labels_df = get_orbit_features(features_file_path, variable_name='out_EM') 
    labels = pd.Series(labels_df['Orbit Family']).repeat(5).reset_index(drop=True).to_numpy()

    # Load orbit data
    orbit_data = load_orbit_data(orbits_file_path, dataset_path='/files/PERIODIC ORBITS')
    reshaped_array = orbit_data.reshape(36071, 7, 5, 1500)
    orbit_data_final = reshaped_array.transpose(0, 2, 1, 3).reshape(36071 * 5, 7, 1500)

    # Save the data if an output file path is provided
    if file_extension == '.npy':
        np.save(output_file_path, orbit_data_final)
        np.save(labels_path, labels)
        data_memmap = np.load(output_file_path, mmap_mode='r+')

    return data_memmap, labels

### 1st period

In [ ]:
#| export
def get_1p_em_dataset(data_directory: Optional[str] = '../data',
                      output_file_path: Optional[str] = '../data/1p_dataset_em.npy') -> Tuple[np.memmap, np.ndarray]:
    """
    Load orbit data from an HDF5 file and corresponding labels from a MAT file, and optionally save the loaded data.
    If the output file exists, it returns the data as a memory-mapped array. After saving, the data is also returned as a memory-mapped array.
    """
    # Extract file extension to determine the type
    _, file_extension = os.path.splitext(output_file_path)

    # Validate supported file types
    if file_extension not in ['.hdf5', '.npy']:
        raise ValueError("Unsupported file extension. Supported extensions are '.hdf5' or '.npy'.")

    # Define the path for labels file
    labels_path = f"{os.path.splitext(output_file_path)[0]}_labels.npy"

    # Check if the combined data file already exists
    if os.path.exists(output_file_path):
        data_memmap = np.load(output_file_path, mmap_mode='r+') if file_extension == '.npy' else None
        labels_memmap = np.load(labels_path)
        return data_memmap, labels_memmap

    # Paths to data files
    orbits_file_path = os.path.join(data_directory, "em_orbits.h5")
    features_file_path = os.path.join(data_directory, "em_features.mat")

    # Load orbit labels
    labels_df = get_orbit_features(features_file_path, variable_name='out_EM') 
    labels = labels_df['Orbit Family'].to_numpy()

    # Load orbit data
    orbit_data = load_orbit_data(orbits_file_path, dataset_path='/files/PERIODIC ORBITS')
    reshaped_orbit_data = orbit_data[:, :, :1500]

    # Save the data if an output file path is provided
    if file_extension == '.npy':
        np.save(output_file_path, reshaped_orbit_data)
        np.save(labels_path, labels)
        data_memmap = np.load(output_file_path, mmap_mode='r+')

    return data_memmap, labels

### Specific Period for each Orbit

In [ ]:
#| export
def get_sp_em_dataset(data_directory: Optional[str] = '../data', 
                      output_file_path: Optional[str] = '../data/sp_dataset_em.npy') -> Tuple[np.memmap, np.ndarray]:
    """
    Load orbit data and corresponding labels based on the specified number of periods per orbit.
    If the output file exists, return the data as a memory-mapped array, otherwise process and save the data.
    After saving, the data is also returned as a memory-mapped array.
    """
    # Extract file extension to determine the type
    _, file_extension = os.path.splitext(output_file_path)

    # Validate supported file types
    if file_extension not in ['.hdf5', '.npy']:
        raise ValueError("Unsupported file extension. Supported extensions are '.hdf5' or '.npy'.")

    # Define the path for labels file
    labels_path = f"{os.path.splitext(output_file_path)[0]}_labels.npy"

    # Check if the combined data file already exists
    if os.path.exists(output_file_path):
        data_memmap = np.load(output_file_path, mmap_mode='r+') if file_extension == '.npy' else None
        labels_memmap = np.load(labels_path)
        return data_memmap, labels_memmap

    # Paths to data files
    orbits_file_path = os.path.join(data_directory, "em_orbits.h5")
    features_file_path = os.path.join(data_directory, "em_features.mat")
    periods_file_path = os.path.join(data_directory, "em_periods.npy")

    # Load orbit labels
    labels_df = get_orbit_features(features_file_path, variable_name='out_EM') 
    labels = labels_df['Orbit Family'].to_numpy()

    # Load number of periods per orbit
    periods_per_orbit = np.load(periods_file_path)

    # Load orbit data
    with h5py.File(orbits_file_path, 'r') as file:
        orbit_data = np.array(file['/files/PERIODIC ORBITS'])

    # Initialize the result array and label list
    orbit_data_final = []
    final_labels = []

    # Iterate over the orbits and periods array
    for index, num_periods in enumerate(periods_per_orbit):
        for period_idx in range(num_periods):
            start_idx = 1500 * period_idx
            end_idx = start_idx + 1500
            orbit_slice = orbit_data[index, :, start_idx:end_idx]
            orbit_data_final.append(orbit_slice)
            final_labels.append(labels[index])

    # Convert list to numpy array
    orbit_data_final = np.stack(orbit_data_final)  # This ensures a uniform 3D array

    # Save the data if an output file path is provided
    if file_extension == '.npy':
        np.save(output_file_path, orbit_data_final)
        np.save(labels_path, final_labels)
        data_memmap = np.load(output_file_path, mmap_mode='r+')
        final_labels = np.load(labels_path)

    return data_memmap, final_labels

In [ ]:
# data, labels = get_sp_em_dataset()

## Fixed time step

### Specific Period for each Orbit

include labels

In [ ]:
#| export
def get_sp_fixed_em_dataset(data_directory: Optional[str] = '../data', 
                            output_data_path: Optional[str] = '../data/em_periods.npy', 
                            output_periods_path: Optional[str] = '../data/em_periods.npy') -> Tuple[Dict[int, np.ndarray], np.ndarray]:
    """
    Load orbit data from an HDF5 file.
    """

    # Define the default data directory based on the script's location if not provided
    if data_directory is None:
        data_directory = os.path.join(os.path.dirname(__file__), "Data")
    
    # Define the file path for the orbit data
    file_path = os.path.join(data_directory, "em_orbits_dt_0_01.h5")
    
    # Initialize a dictionary to hold data from all datasets
    orbit_data = {}
    
    # Open the HDF5 file and read data from each dataset
    with h5py.File(file_path, 'r') as file:
        # Iterate through each dataset in the HDF5 file
        for name in file:
            dataset = np.array(file[name])
            if name.isdigit():
                orbit_data[int(name)] = dataset
            else:
                orbit_data[name] = dataset
    
    # Extract the periods
    periods = orbit_data.get('prop_periods')[0] if 'prop_periods' in orbit_data else None

    # Save the entire orbit data if an output path is provided
    if output_data_path:
        if output_data_path.endswith('.npy'):
            np.save(output_data_path, orbit_data)
        elif output_data_path.endswith('.h5'):
            with h5py.File(output_data_path, 'w') as h5_file:
                for key, data in orbit_data.items():
                    h5_file.create_dataset(key, data=data)

    # Save periods data if an output path is provided
    if output_periods_path:
        if output_periods_path.endswith('.npy'):
            np.save(output_periods_path, periods)
        elif output_periods_path.endswith('.h5'):
            with h5py.File(output_periods_path, 'w') as h5_file:
                h5_file.create_dataset('periods', data=periods)

    return orbit_data, periods

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()